# 10 notebook that applies all project cards (replacing notebooks 02-09)

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_11')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_11')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-06-22 11:32:37, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 11:32:37, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 11:32:37, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 11:32:37, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


# Read pickle

In [6]:
version_01_pickle_file_name = os.path.join(input_dir, 'working_scenario_01.pickle')
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

In [7]:
v_01_scenario.transit_net.feed.stop_times.columns

Index(['arrival_time', 'departure_time', 'stop_sequence', 'stop_headsign',
       'pickup_type', 'drop_off_type', 'shape_dist_traveled', 'timepoint',
       'start_service_area_id', 'end_service_area_id',
       'start_service_area_radius', 'end_service_area_radius',
       'continuous_pickup', 'continuous_drop_off', 'pickup_area_id',
       'drop_off_area_id', 'pickup_service_area_radius',
       'drop_off_service_area_radius', 'last_stop_on_trip', 'trip_id',
       'stop_id'],
      dtype='object')

In [100]:
v_01_scenario.road_net.links_df[v_01_scenario.road_net.links_df.model_link_id == 2186785]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,locationReferences,shape_id,geometry,ft_cal,ft,useclass,assignable_cal,assignable,transit,id
593106,nan,True,True,934fc254e3778809baa433a1f982b9f9,1,nan,Leigh Avenue,False,nan,secondary,...,"[{'sequence': 1, 'point': [-121.92268, 37.2425...",738152,"LINESTRING (-121.92268 37.24257, -121.92267 37...",6,6,0,1,1,0,86fe41c9709c917d632a302dd707e825


# Create scenario and apply projects

waiting on dependencies to work

In [8]:
v_01_scenario.road_net.links_df["id"] = v_01_scenario.road_net.links_df["shstGeometryId"]

In [9]:
step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory = card_dir,
    tags = ['highway review', "Major Arterial Review", "Reversible Lanes", 'Bus Only', 'Toll Plaza'],
    validate_project_cards=False)

2021-06-22 11:33:58, INFO: Creating Scenario
2021-06-22 11:33:58, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [10]:
step1_scenario.apply_all_projects()

2021-06-22 11:34:26, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-06-22 11:34:26, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-06-22 11:34:26, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-06-22 11:34:26, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-06-22 11:34:26, INFO: Applying Project to Roadway Network: Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-06-22 11:34:26, INFO: Applying Project to Roadway Network: Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-06-22 11:34:27, INFO: Applying add san mateo bridge toll plaza
2021-06-22 11:34:27, INFO: Applying add san mateo bridge toll plaza
2021-06-22 11:34:27, INFO: Applying add san mateo bridge toll plaza
2021-06-22 11:34:27, INFO: Applying add san mateo bridge toll plaza
2021-06-22 11:34:27, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza
2021-06-22 11:34:27, INFO: Applying 

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-06-22 11:34:28, INFO: Applying Project to Transit Network: add san mateo bridge toll plaza
2021-06-22 11:34:28, INFO: Applying Project to Transit Network: add san mateo bridge toll plaza
2021-06-22 11:34:28, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2021-06-22 11:34:28, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2021-06-22 11:34:28, INFO: Applying add san mateo bridge toll plaza
2021-06-22 11:34:28, INFO: Applying add san mateo bridge toll plaza
2021-06-22 11:34:28, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza
2021-06-22 11:34:28, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-06-22 11:36:19, INFO: Applying Year 2015 Correct As Is
2021-06-22 11:36:19, INFO: Applying Year 2015 Correct As Is
2021-06-22 11:36:19, INFO: Applying Year 2015 Correct As Is
2021-06-22 11:36:19, INFO: Applying Year 2015 Correct As Is
2021-06-22 11:36:19, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-06-22 11:36:19, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-06-22 11:36:20, INFO: Applying Year 2015 Modify Facility Type to Service
2021-06-22 11:36:20, INFO: Applying Year 2015 Modify Facility Type to Service
2021-06-22 11:36:20, INFO: Applying Year 2015 Modify Facility Type to Service
2021-06-22 11:36:20, INFO: Applying Year 2015 Modify Facility Type to Service
2021-06-22 11:36:20, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service
2021-06-22 11:36:20, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service
2021-06-22 11:36:20, INFO: Applying Year 2015 Modify Facilit

2021-06-22 11:36:23, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-06-22 11:36:23, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-06-22 11:36:23, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-06-22 11:36:23, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-06-22 11:36:23, INFO: Applying Project to Roadway Network: Year 2015 Should be 5 Lane correct facility type
2021-06-22 11:36:23, INFO: Applying Project to Roadway Network: Year 2015 Should be 5 Lane correct facility type
2021-06-22 11:36:23, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-06-22 11:36:23, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-06-22 11:36:23, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-06-22 11:36:23, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-06-22 11:36:23, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 5 Lanes
2021-06-22 11:36:23, INFO: Applying Project to Roadway N

2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-06-22 11:36:27, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 1 Lane
2021-06-22 11:36:27, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 1 Lane
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-06-22 11:36:27, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-06-22 11:36:27, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 14 Lanes
2021-06-22 11:36:27, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 14 Lanes
2021-06-22 11:36:27, INFO: Applying Year 2015 Sh

2021-06-22 11:40:15, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-06-22 11:40:15, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-06-22 11:40:15, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-06-22 11:40:15, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-06-22 11:40:15, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-06-22 11:40:15, INFO: Applying year 2015 sterling on ramp HOV only in PM
2021-06-22 11:40:15, INFO: Applying year 2015 sterling on ramp HOV only in PM
2021-06-22 11:40:15, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2021-06-22 11:40:15, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2021-06-22 11:40:15, INFO: Applying year 2015 golden gate reversible lanes
2021-06-22 

In [11]:
step1_scenario.road_net.links_df[step1_scenario.road_net.links_df.model_link_id == 8063198][["A", "B", "geometry", "id"]]

,A,B,geometry,id
1634773,5013515,5030001,"LINESTRING (-122.48221 37.83307, -122.48175 37...",d01b27cabff869e0bf4f7187ee77d6de


In [12]:
step1_scenario.road_net.nodes_df[step1_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644476,NaN,,Marin,1,0,0,5030001,0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [13]:
list(step1_scenario.road_net.links_df[step1_scenario.road_net.links_df.model_link_id == 8063198].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [14]:
list(step1_scenario.road_net.shapes_df[step1_scenario.road_net.shapes_df.id == "d01b27cabff869e0bf4f7187ee77d6de"].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [15]:
step2_scenario = Scenario.create_scenario(
    base_scenario = step1_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['Managed Lanes', 'toll review', 'Exclude Trucks'],
    validate_project_cards=False)

2021-06-22 11:46:23, INFO: Creating Scenario
2021-06-22 11:46:23, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [16]:
step2_scenario.apply_all_projects()

2021-06-22 11:47:24, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 11:47:24, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 11:47:24, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 11:47:24, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 11:47:24, INFO: Applying year 2015 modify hov lanes and use on san mateo bridge t

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-06-22 11:47:25, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-06-22 11:47:25, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-06-22 11:47:25, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-06-22 11:47:25, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-06-22 11:47:25, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-06-22 11:47:25, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-06-22 11:47:27, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2021-06-22 11:47:27, INFO: Applying Year 2015 Modify HOV

2021-06-22 11:48:18, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-06-22 11:48:18, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-06-22 11:48:18, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-06-22 11:48:18, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-06-22 11:48:36, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-06-22 11:48:36, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-06-22 11:48:36, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-06-22 11:48:36, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-06-22 11:48:36, INFO: Apply

2021-06-22 11:49:35, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2021-06-22 11:49:35, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2021-06-22 11:49:35, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2021-06-22 11:49:35, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2021-06-22 11:49:35, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2021-06-22 11:49:35, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2021-06-22 11:49:38, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Carquinez Bridge to Bay Bridge Toll Plaza
2021-06-22 11:49:38, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Carquinez Bridge to Bay Bridge Toll Plaza
2021-06-22 11:49:38, INFO: Applying Year 2015 Modi

2021-06-22 11:50:10, INFO: Applying Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-06-22 11:50:10, INFO: Applying Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-06-22 11:50:10, INFO: Applying Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-06-22 11:50:10, INFO: Applying Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-06-22 11:50:10, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-06-22 11:50:10, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-06-22 11:50:11, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-06-22 11:50:11, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-06-22 11:50:11, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-06-22 11:50:11, INFO: Applying Year 2015 Modify USE on I-580

2021-06-22 11:50:18, INFO: Applying Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-06-22 11:50:18, INFO: Applying Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-06-22 11:50:18, INFO: Applying Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-06-22 11:50:18, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-06-22 11:50:18, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-06-22 11:50:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2021-06-22 11:50:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2021-06-22 11:50:22, INFO: Applying Year 

2021-06-22 11:51:13, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-06-22 11:51:13, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-06-22 11:51:13, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-06-22 11:51:13, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-06-22 11:51:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-06-22 11:51:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-06-22 11:51:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-06-22 11:51:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-06-22 11:51:16, INFO: Applying Project to Roadw

2021-06-22 11:51:21, INFO: Applying year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass
2021-06-22 11:51:21, INFO: Applying year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass
2021-06-22 11:51:21, INFO: Applying Project to Roadway Network: year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass
2021-06-22 11:51:21, INFO: Applying Project to Roadway Network: year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass
2021-06-22 11:51:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on Dumbarton Bridge Toll Plaza Toll Plaza on CA-84 WB - Bridge Pass
2021-06-22 11:51:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on Dumbarton Bridge Toll Plaza Toll Plaza on CA-84 WB - Bridge Pass
2021-06-22 11:51:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on Dumbarton Bridge Toll Plaza Toll Plaza on CA-84 WB - Bridge Pass
2021-06-22 11:51:22, INFO: Applying

In [17]:
step2_scenario.road_net.links_df.shape

(1634775, 45)

In [18]:
step2_scenario.road_net.links_df.model_link_id.nunique()

1634775

In [19]:
step2_scenario.road_net.links_df.county.isnull().sum()

0

In [20]:
step2_scenario.road_net.links_df.model_link_id.max()

8063199

In [21]:
step3_scenario = Scenario.create_scenario(
    base_scenario= step2_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Major Transit links"],
    validate_project_cards=False)

step3_scenario.apply_all_projects()

2021-06-22 11:56:22, INFO: Creating Scenario
2021-06-22 11:56:22, INFO: Creating Scenario
2021-06-22 11:56:26, INFO: Applying move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 11:56:26, INFO: Applying Project to Roadway Network: move street rail to street connector links


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-06-22 11:58:09, INFO: Applying move street rail to street connector links
2021-06-22 11:58:09, INFO: Applying move street rail to street connector links
2021-06-22 11:58:09, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 11:58:09, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 11:59:52, INFO: Applying move street rail to street connector links
2021-06-22 11:59:52, INFO: Applying move street rail to street connector links
2021-06-22 11:59:52, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 11:59:52, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-06-22 12:01:34, INFO: Applying move street rail to street connector links
2021-06-22 12:01:34, INFO: Applying move street rail to street connector links
2021-06-22 12:01:34, INFO: Applying Project to Roadway Network: move street rail to street connect

In [22]:
step3_scenario.road_net.links_df.shape

(1634793, 45)

In [23]:
step3_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry', 'ft_cal', 'ft',
       'useclass', 'assignable_cal', 'assignable', 'transit', 'id',
       'tollbooth', 'bus_only', 'transit_access', 'managed', 'ML_lanes',
       'segment_id', 'ML_tollbooth', 'ML_useclass', 'ML_access', 'ML_egress',
       'ML_tollseg', 'tollseg'],
      dtype='object')

In [24]:
step3_scenario.transit_net.set_roadnet(step3_scenario.road_net, 
                                      validate_consistency = False)

step4_scenario = Scenario.create_scenario(
    base_scenario= step3_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Minor Transit", "Add Transit", "Major Transit", "Toll Plaza Transit"],
    validate_project_cards=False)

step4_scenario.apply_all_projects()

2021-06-22 12:37:05, INFO: Creating Scenario
2021-06-22 12:37:05, INFO: Creating Scenario
2021-06-22 12:40:24, ERROR: Projects sfmta route n-judah has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-06-22 12:40:24, ERROR: Projects sfmta route n-judah has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-06-22 12:40:24, ERROR: Projects update routing for golden gate bridge toll plaza transit has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 12:40:24, ERROR: Projects update routing for golden gate bridge toll plaza transit has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 12:40:24, ERROR: Projects update routing for san mateo bridge toll plaza transit has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2021-06-22 12:40:24,

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2161338
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2161338
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2146003
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2146003
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2064421
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2064421
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-06-22 12:40:52, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-06-22 12:40:54, INFO: A

2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2131220
2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2043788
2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2043788
2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2108685
2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2108685
2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2046425
2021-06-22 12:40:57, INFO: Creating a new stop in stops.txt for node ID: 2046425
2021-06-22 12:40:58, INFO: Creating a new stop in stops.txt for node ID: 2098451
2021-06-22 12:40:58, INFO: Creating a new stop in stops.txt for node ID: 2098451
2021-06-22 12:40:58, INFO: Creating a new stop in stops.txt for node ID: 2020562
2021-06-22 12:40:58, INFO: Creating a new stop in stops.txt for node ID: 2020562
2021-06-22 12:40:58, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-06-22 12:40:58, INFO: C

2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2120028
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2030540
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2030540
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2075183
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2075183
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2002834
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2002834
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2001551
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2001551
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2048716
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2048716
2021-06-22 12:41:03, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-06-22 12:41:03, INFO: C

...

2021-06-22 12:43:19, INFO: Applying SFMTA Route 349 Edits
2021-06-22 12:43:19, INFO: Applying SFMTA Route 349 Edits
2021-06-22 12:43:19, INFO: Applying Project to Transit Network: SFMTA Route 349 Edits
2021-06-22 12:43:19, INFO: Applying Project to Transit Network: SFMTA Route 349 Edits
2021-06-22 12:43:20, INFO: Applying SFMTA Route 346 Edits
2021-06-22 12:43:20, INFO: Applying SFMTA Route 346 Edits
2021-06-22 12:43:20, INFO: Applying SFMTA Route 346 Edits
2021-06-22 12:43:20, INFO: Applying SFMTA Route 346 Edits
2021-06-22 12:43:20, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-06-22 12:43:20, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-06-22 12:43:22, INFO: Applying SFMTA Route 346 Edits
2021-06-22 12:43:22, INFO: Applying SFMTA Route 346 Edits
2021-06-22 12:43:22, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-06-22 12:43:22, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-06-22 12:43:23,

2021-06-22 12:43:49, INFO: Applying SFMTA Route 306 Edits
2021-06-22 12:43:49, INFO: Applying SFMTA Route 306 Edits
2021-06-22 12:43:49, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-06-22 12:43:49, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-06-22 12:43:50, INFO: Creating a new stop in stops.txt for node ID: 1012353
2021-06-22 12:43:50, INFO: Creating a new stop in stops.txt for node ID: 1012353
2021-06-22 12:43:50, INFO: Creating a new stop in stops.txt for node ID: 1020811
2021-06-22 12:43:50, INFO: Creating a new stop in stops.txt for node ID: 1020811
2021-06-22 12:43:51, INFO: Applying SFMTA Route 306 Edits
2021-06-22 12:43:51, INFO: Applying SFMTA Route 306 Edits
2021-06-22 12:43:51, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-06-22 12:43:51, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-06-22 12:43:52, INFO: Applying SFMTA Route 306 Edits
2021-06-22 12:43:52, INFO: Applying SFMTA 

2021-06-22 12:57:45, INFO: Creating a new stop in stops.txt for node ID: 1516920
2021-06-22 12:57:45, INFO: Creating a new stop in stops.txt for node ID: 1516920
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1525498
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1525498
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1516256
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1516256
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1513094
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1513094
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1513976
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1513976
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1528479
2021-06-22 12:57:46, INFO: Creating a new stop in stops.txt for node ID: 1528479
2021-06-22 12:57:46, INFO: C

2021-06-22 12:57:53, INFO: Applying Project to Transit Network: BART Fremont Daly City reverse shape
2021-06-22 12:57:54, INFO: Applying BART Fremont Daly City reverse shape
2021-06-22 12:57:54, INFO: Applying BART Fremont Daly City reverse shape
2021-06-22 12:57:54, INFO: Applying Project to Transit Network: BART Fremont Daly City reverse shape
2021-06-22 12:57:54, INFO: Applying Project to Transit Network: BART Fremont Daly City reverse shape
2021-06-22 12:57:55, INFO: Applying year 2015 ac transit route 1r
2021-06-22 12:57:55, INFO: Applying year 2015 ac transit route 1r
2021-06-22 12:57:55, INFO: Applying year 2015 ac transit route 1r
2021-06-22 12:57:55, INFO: Applying year 2015 ac transit route 1r
2021-06-22 12:57:55, INFO: Applying Project to Transit Network: year 2015 ac transit route 1r
2021-06-22 12:57:55, INFO: Applying Project to Transit Network: year 2015 ac transit route 1r
2021-06-22 12:57:56, INFO: Creating a new stop in stops.txt for node ID: 2502166
2021-06-22 12:57:5

2021-06-22 12:58:08, INFO: Applying year 2015 ac transit route 38 edits
2021-06-22 12:58:08, INFO: Applying Project to Transit Network: year 2015 ac transit route 38 edits
2021-06-22 12:58:08, INFO: Applying Project to Transit Network: year 2015 ac transit route 38 edits
2021-06-22 12:58:09, INFO: Creating a new stop in stops.txt for node ID: 2503531
2021-06-22 12:58:09, INFO: Creating a new stop in stops.txt for node ID: 2503531
2021-06-22 12:58:09, INFO: Creating a new stop in stops.txt for node ID: 2539453
2021-06-22 12:58:09, INFO: Creating a new stop in stops.txt for node ID: 2539453
2021-06-22 12:58:09, INFO: Creating a new stop in stops.txt for node ID: 2564896
2021-06-22 12:58:09, INFO: Creating a new stop in stops.txt for node ID: 2564896
2021-06-22 12:58:10, INFO: Applying year 2015 ac transit route 06 edits
2021-06-22 12:58:10, INFO: Applying year 2015 ac transit route 06 edits
2021-06-22 12:58:10, INFO: Applying year 2015 ac transit route 06 edits
2021-06-22 12:58:10, INFO:

In [25]:
step4_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = step4_scenario.road_net,
    parameters = parameters)

2021-06-22 13:08:54, INFO: Adding centroid and centroid connector to standard network
2021-06-22 13:08:54, INFO: Adding centroid and centroid connector to standard network
2021-06-22 13:08:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 13:08:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 13:08:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 13:08:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 13:09:09, INFO: Finished adding centroid and centroid connectors
2021-06-22 13:09:09, INFO: Finished adding centroid and centroid connectors


In [26]:
print(step4_scenario.road_net.links_df.shape)
print(step4_scenario.road_net.links_df.model_link_id.nunique())
print(step4_scenario.road_net.nodes_df.shape)
print(step4_scenario.road_net.nodes_df.model_node_id.nunique())
print(step4_scenario.road_net.shapes_df.shape)

(2199295, 45)
2199295
(688959, 12)
688959
(1152249, 7)


In [27]:
step4_scenario.road_net.links_df.locationReferences.iloc[-1]

[{'sequence': 1,
  'point': [-122.50580293859502, 37.94830473568058],
  'distanceToNextRef': 51.382886808196375,
  'bearing': 0,
  'intersectionId': nan},
 {'sequence': 2,
  'point': [-122.50576480000001, 37.9487303],
  'intersectionId': '44c53879b5a9797cbb2eff72b5f64699'}]

In [28]:
step5_scenario = Scenario.create_scenario(
    base_scenario= step4_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['External Stations Review'],
    validate_project_cards=False)

step5_scenario.apply_all_projects()

2021-06-22 13:12:43, INFO: Creating Scenario
2021-06-22 13:12:43, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


2021-06-22 13:16:28, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-06-22 13:16:28, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-06-22 13:16:28, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-06-22 13:16:28, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-06-22 13:16:28, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-06-22 13:16:28, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-06-22 13:18:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-06-22 13:18:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-06-22 13:18:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-06-22 13:18:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-06-22 13:18:47, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-06-22 13:18:47, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-06-22 13:21:04, INFO: Applying Year 2015 Add centroid connector at External Station connecting Vic Fazio Highway CA-1

2021-06-22 13:41:49, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-06-22 13:41:49, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-06-22 13:41:49, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-06-22 13:41:49, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-06-22 13:41:49, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-06-22 13:41:49, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-06-22 13:44:05, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 SB
2021-06-22 13:44:05, INFO: Applying Year 2015 Add centroi

2021-06-22 14:04:40, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-06-22 14:04:40, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-06-22 14:04:40, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-06-22 14:04:40, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-06-22 14:06:57, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-06-22 14:06:57, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-06-22 14:06:57, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-06-22 14:06:57, INFO

In [29]:
print(step5_scenario.road_net.links_df.shape)
print(step5_scenario.road_net.links_df.model_link_id.nunique())
print(step5_scenario.road_net.nodes_df.shape)
print(step5_scenario.road_net.nodes_df.model_node_id.nunique())

(2199321, 45)
2199321
(688959, 12)
688959


# sanity check

In [30]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3150085] # should be ramp lanes 2

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
765902,,False,True,d1b76ff92ba7a2915de8efc97c782d73,2,,,True,,motorway_link,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3107408] # bay bridge toll link

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
548523,"['nan', 'nan']",False,True,826a99158fd58e5483d1085c9e0d65b3,"{'default': 4, 'timeofday': [{'time': (21600, ...","['nan', 'nan']","['nan', 'nan']","[True, True]","['I 80', 'I 80']",motorway,...,NaN,1,"{'default': 0, 'timeofday': [{'time': (21600, ...",5,5,"{'default': 0, 'timeofday': [{'time': (21600, ...",all,all,NaN,NaN


In [32]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2199301, dtype: object

In [33]:
# this link is splitted into two for toll plaza at GGB

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8016300]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg


In [34]:
# golden gate bridge

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063198].lanes.iloc[0]

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 4},
  {'time': (54000, 68400), 'value': 2}]}

In [35]:
# golden gate bridge HOV

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063198].ML_lanes.iloc[0]

'NaN'

In [36]:
# check if new node is created for toll plaza at GGB

step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644476,NaN,,Marin,1,0,0,5030001,0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [37]:
# golden gate bridge toll plaza

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063199].lanes.iloc[0]

{'default': 9,
 'timeofday': [{'time': (21600, 36000), 'value': 7},
  {'time': (54000, 68400), 'value': 7}]}

In [38]:
# check shapes for new links
# example: toll plaza at GGB and a new rail-to-street link

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin([8063198, 8063199, 26067378])][["id", "geometry"]]

,id,geometry
1634773,d01b27cabff869e0bf4f7187ee77d6de,"LINESTRING (-122.48221 37.83307, -122.48175 37..."
1634774,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
1634775,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [39]:
step5_scenario.road_net.shapes_df[
    step5_scenario.road_net.shapes_df.id.isin(["b46d3b9a4683bfe26b97e2a66181d8dd", "ba0e166e3a9f6c1cc8d7e331415769b2"])][
        ["id", "geometry"]]

,id,geometry
869975,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
869976,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [40]:
step5_scenario.transit_net.feed.shapes.shape_id.nunique()

1592

In [41]:
step5_scenario.transit_net.feed.shapes[
    step5_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,176,65,65,176,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


In [42]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.A.isin([2579949, 2607569])][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
514595,2579949,2535246,residential,0
842601,2579949,2604856,residential,1
846406,2607569,2586861,residential,1
1009927,2607569,2546185,residential,1
1077889,2607569,2541364,residential,0
1645012,2579949,300963,taz,1
1645930,2607569,300963,taz,1
1820104,2579949,318103,maz,1
1820105,2579949,327248,maz,1
1820106,2579949,328044,maz,1


In [43]:
# check Service Road Dumbarton Bridge Bypass is not assignable
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 3152465][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
778002,2552715,2540313,motorway_link,0


In [44]:
# check should be 3 lanes

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 2217343][["A", "B", "roadway","assignable", "lanes"]]

,A,B,roadway,assignable,lanes
690625,2153360,2071468,motorway,1,"{'default': 3, 'timeofday': [{'time': (21600, ..."


In [45]:
# should be ramps
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
        [3031886,3006561,3046803,3101610,3011518,3024387,4018032])][["A", "B", "roadway","assignable", "ft", "walk_access"]]

,A,B,roadway,assignable,ft,walk_access
33757,2506600,2542798,motorway_link,1,3,1
59112,2511462,2523557,motorway_link,1,3,1
123885,3017507,3058438,motorway_link,1,3,1
124668,2523557,2549665,motorway_link,1,3,1
162620,2501507,2506600,motorway_link,1,3,1
238525,2542798,2518014,motorway_link,1,3,1
518512,2580420,2511462,motorway_link,1,3,1


In [62]:
# zone id should be "2" (ac transit), "1" (dumbarton union city), "2" (ac transit dumbarton), "1" (samtrans) , "1" (samtrans)

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["101523872", "4694", "4669", "102036044", "4187"])]

,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,stop_code,platform_code,position,direction,stop_id,osm_node_id,shst_node_id,model_node_id
19329,Transbay Temp Terminal,NaN,37.789707,-122.393626,2,NaN,NaN,place_ETSF,NaN,NaN,50751,NaN,NaN,NaN,4187,3955000020,459ceb872e72e49fd4041a78884935e2,1021597
19810,Union City BART,NaN,37.590360,-122.017465,1,NaN,NaN,NaN,NaN,NaN,50781,NaN,NaN,NaN,4669,1025879893,84c3f377f9f0065bf14be3af92202d0d,2509987
19835,Stanford Oval,NaN,37.429272,-122.168925,2,NaN,NaN,NaN,NaN,NaN,55432,NaN,NaN,NaN,4694,522171022,6d2aeffcbad07f0873a268bea42838f3,2149155
22309,NaN,NaN,37.444794,-122.168849,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22338,NaN,NaN,37.673730,-122.467029,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872


## Make Travel Model Networks

In [63]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = step5_scenario.road_net, 
    parameters = parameters)

2021-06-22 14:55:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:55:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:55:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:55:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:55:14, INFO: Filling nan for network from network wrangler
2021-06-22 14:55:14, INFO: Filling nan for network from network wrangler
2021-06-22 14:55:47, INFO: Converting variable type to mtc standard
2021-06-22 14:55:47, INFO: Converting variable type to mtc standard


In [64]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2199321, 45)
2199321
(688959, 12)
(1152275, 7)


In [65]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    transit_network = step5_scenario.transit_net, 
    parameters = parameters)

2021-06-22 14:56:39, INFO: Adding tap and tap connector to standard network
2021-06-22 14:56:39, INFO: Adding tap and tap connector to standard network
2021-06-22 14:56:39, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:56:39, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:56:39, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:56:39, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:56:40, INFO: There are 596 stops not connected to taps, connecting them to the closest taps.
2021-06-22 14:56:40, INFO: There are 596 stops not connected to taps, connecting them to the closest taps.
snapping gtfs stops to roadway node osmid...


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2594: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  crs = "EPSG:4326"


2021-06-22 14:56:47, INFO: Finished adding tap and tap connectors
2021-06-22 14:56:47, INFO: Finished adding tap and tap connectors


In [66]:
# Vallejo Ferry to TAP

model_net.links_df[model_net.links_df.B == 3541176][["A", "roadway", "county", "rail_only", "model_link_id"]]

,A,roadway,county,rail_only,model_link_id
191843,3512789,footway,Solano,0.0,5013621
1634794,1028038,,,1.0,26067374
1634796,1028039,,,1.0,26067376
2235683,590188,tap,Solano,NaN,5157453


In [67]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2235749, 45)
2235749
(694959, 12)
694959
(1170489, 7)
1170489


In [68]:
# update farezone due to AC Transit Fare change

model_net = mtc.calculate_farezone(
    roadway_network = model_net,
    transit_network = step5_scenario.transit_net,
    parameters = parameters,
    network_variable = 'farezone',
    overwrite = True,
)

model_net.nodes_df.farezone.value_counts()

2021-06-22 14:57:14, INFO: Determining farezone
2021-06-22 14:57:14, INFO: Determining farezone
2021-06-22 14:57:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:57:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:57:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:57:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 14:57:14, INFO: Overwriting existing Variable 'farezone' already in network
2021-06-22 14:57:14, INFO: Overwriting existing Variable 'farezone' already in network
2021-06-22 14:57:14, INFO: Calculating and adding roadway network variable: farezone
2021-06-22 14:57:14, INFO: Calculating and adding roadway network variable: farezone
2021-06-22 14:57:15, INFO: Finished determining variable: farezone
2021-06-22 14:57:15, INFO: Finished determining variable: farezone


7.0      4208
111.0    3236
90.0     1537
83.0      226
73.0      207
         ... 
260.0       1
259.0       1
258.0       1
257.0       1
244.0       1
Name: farezone, Length: 296, dtype: int64

In [69]:
step5_scenario.transit_net.feed.stops.zone_id.unique()

array(['12TH', '16TH', '19TH', '1', '24TH', 'ASHB', 'BALB', 'BAYF',
       'CAST', 'CIVC', 'COLS', 'COLM', 'CONC', 'DALY', 'DBRK', 'DUBL',
       'DELN', 'PLZA', 'EMBR', 'FRMT', 'FTVL', 'GLEN', 'HAYW', 'LAFY',
       'LAKE', 'MCAR', 'MLBR', 'MONT', 'NBRK', 'NCON', 'OAKL', 'ORIN',
       'PITT', 'PHIL', 'POWL', 'RICH', 'ROCK', 'SBRN', 'SFIA', 'SANL',
       'SHAY', 'SSAN', 'UCTY', 'WCRK', 'WDUB', 'WOAK', '55644', '55645',
       '55449', '55643', '55448', '55450', nan, '2', '3', '4', '5', '6',
       '116', '667', '664', '666', '669', '663', '670', '665', '7', '198',
       '199', '0', '462', '471', '466', '469', '470', '468', '467', '463',
       '472', '461', '550', '464', '465', '473', '460', '483', '485',
       '584', '56138', '56134', '56139', '56143', '56144', '56142',
       '56136', '56141', '56140', '56137', '56133', '56146', '56135',
       '56145'], dtype=object)

In [70]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-06-22 14:57:29, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-06-22 14:57:29, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-06-22 14:57:29, INFO: Creating managed lane network.
2021-06-22 14:57:29, INFO: Creating managed lane network.
2021-06-22 14:57:29, INFO: Creating network with duplicated managed lanes
2021-06-22 14:57:29, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2021-06-22 15:12:26, INFO: Determining cntype
2021-06-22 15:12:26, INFO: Determining cntype
2021-06-22 15:12:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 15:12:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 15:12:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 15:12:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 15:12:26, INFO: Calculating and adding roadway network variable: cntype
2021-06-22 15:12:26, INFO: Calculating and adding roadway network variable: cntype
2021-06-22 15:14:01, INFO: Finished determining variable: cntype
2021-06-22 15:14:01, INFO: Finished determining variable: cntype
2021-06-22 15:14:01, INFO: Determining transit
2021-06-22 15:14:01, INFO: Determining transit
2021-06-22 15:14:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 15:14:01, INFO: Lasso base directory set a

In [71]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2240148, 56)
2240148
(696467, 14)
696467
(1174899, 8)
1174899


In [72]:
model_net.links_df.rail_only.value_counts()

0    2239383
1        765
Name: rail_only, dtype: int64

In [73]:
model_net.links_df[(model_net.links_df.bus_only == 1)].cntype.unique()

array(['TANA'], dtype=object)

In [74]:
model_net.links_df.assignable.value_counts()

0    1282911
1     957237
Name: assignable, dtype: int64

In [75]:
model_net.links_df[model_net.links_df.cntype == "TAZ"].assignable.value_counts()

1    32986
Name: assignable, dtype: int64

In [76]:
model_net.links_df[model_net.links_df.cntype == "MAZ"].assignable.value_counts()

1    531534
Name: assignable, dtype: int64

In [77]:
model_net.links_df[model_net.links_df.A == 2601773].roadway

1284           motorway
4220          ml_access
803691    motorway_link
Name: roadway, dtype: object

In [78]:
model_net.links_df[model_net.links_df.model_link_id == 3152465].shstReferenceId

782542    e5742b085ba3dcb41da0c499cecc33bc
Name: shstReferenceId, dtype: object

In [79]:
# AC Transit node farezone
model_net.nodes_df[model_net.nodes_df.model_node_id == 1011480].farezone
model_net.nodes_df.farezone.max()

296

# Write to Disk

In [80]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", "cntype", "lanes_AM", "assignable"],
    node_output_variables = ["model_node_id", "geometry", "farezone", "tap_id"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-06-22 15:24:11, INFO: Writing Network as Shapefile
2021-06-22 15:24:11, INFO: Writing Network as Shapefile
2021-06-22 15:24:11, INFO: Renaming DBF Node Variables
2021-06-22 15:24:11, INFO: Renaming DBF Node Variables
2021-06-22 15:24:11, INFO: Renaming variables so that they are DBF-safe
2021-06-22 15:24:11, INFO: Renaming variables so that they are DBF-safe
2021-06-22 15:24:22, INFO: Renaming DBF Link Variables
2021-06-22 15:24:22, INFO: Renaming DBF Link Variables
2021-06-22 15:24:22, INFO: Renaming variables so that they are DBF-safe
2021-06-22 15:24:22, INFO: Renaming variables so that they are DBF-safe
2021-06-22 15:24:25, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_11\nodes.shp
2021-06-22 15:24:25, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_11\nodes.shp
2021-06-22 15:25:19, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

In [81]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-06-22 15:32:27, INFO: Starting fixed width conversion
2021-06-22 15:32:27, INFO: Starting fixed width conversion
2021-06-22 15:56:34, INFO: Writing out link database
2021-06-22 15:56:34, INFO: Writing out link database
2021-06-22 15:56:49, INFO: Writing out link header and width ----
2021-06-22 15:56:49, INFO: Writing out link header and width ----
2021-06-22 15:56:49, INFO: Starting fixed width conversion
2021-06-22 15:56:49, INFO: Starting fixed width conversion
2021-06-22 15:59:04, INFO: Writing out node database
2021-06-22 15:59:04, INFO: Writing out node database
2021-06-22 15:59:06, INFO: Writing out node header and width
2021-06-22 15:59:06, INFO: Writing out node header and width


In [82]:
%%capture
os.chdir(output_dir)
!make_cube.bat

links.txt(110000): 2519866;2611009;0;1;0;      Alameda;   0.03266305781091407
links.txt(111000): 3507299;3507911;1;1;0;       Solano;   0.04795442543995559


## As transit line file (for CUBE)

In [101]:
mtc.write_cube_fare_files(
    roadway_network=model_net,
    transit_network=step5_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

In [84]:
standard_transit_net = StandardTransit.fromTransitNetwork(step5_scenario.transit_net, parameters = parameters)

2021-06-22 16:00:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 16:00:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 16:00:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-06-22 16:00:54, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [85]:
standard_transit_net.feed.routes[standard_transit_net.feed.routes.route_id=="712"]

,route_id_original,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,agency_raw_name,route_sort_order,min_headway_minutes,eligibility_restricted,continuous_pickup,continuous_drop_off,route_id
701,brisbane,alliance,NaN,Brisbane-Bayshore Caltrain,NaN,3,http://commute.org/index.php?option=com_conten...,NaN,NaN,commuteDOTorg_GTFSImportExport_20160127_final_mj,NaN,NaN,NaN,NaN,NaN,712


In [86]:
standard_transit_net.feed.routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 16 columns):
route_id_original         721 non-null object
agency_id                 624 non-null object
route_short_name          666 non-null object
route_long_name           699 non-null object
route_desc                14 non-null object
route_type                721 non-null int64
route_url                 236 non-null object
route_color               260 non-null object
route_text_color          243 non-null object
agency_raw_name           721 non-null object
route_sort_order          95 non-null object
min_headway_minutes       32 non-null object
eligibility_restricted    4 non-null object
continuous_pickup         4 non-null object
continuous_drop_off       4 non-null object
route_id                  721 non-null object
dtypes: int64(1), object(15)
memory usage: 90.2+ KB


In [102]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2021-06-24 15:58:04, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:04, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:14, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:14, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:15, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:15, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:42, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:42, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:49, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 15:58:49, INFO: Converting GTFS Standard Pr

In [103]:
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-06-24 16:06:57, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-06-24 16:06:57, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


## As pickle

In [89]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_11.pickle')
pickle.dump(step5_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [90]:
step5_scenario.road_net.write(path = output_dir, filename = "v11")
step5_scenario.transit_net.write(path = output_dir, filename = "v11")

2021-06-22 16:30:34, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_11
2021-06-22 16:30:34, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_11


In [91]:
# save out link feather
# to_feather() does not like mixed type

links_feather = step5_scenario.road_net.links_df.copy()
links_feather.drop("geometry", axis = 1, inplace = True)

for c in links_feather.columns:
    if c in parameters.int_col:
        links_feather[c] = links_feather[c].astype(int)
    else:
        links_feather[c] = links_feather[c].astype(str)
        
links_feather.to_feather(path = os.path.join(output_dir, "v11_links.feather"))

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\ipykernel_launcher.py:13: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  del sys.path[0]


In [96]:
stop_node_id = step5_scenario.transit_net.feed.stops.model_node_id.tolist()

In [97]:
step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id.isin(stop_node_id)].walk_access.value_counts()

1    18214
Name: walk_access, dtype: int64

In [108]:
model_net.links_df[model_net.links_df.model_link_id == 3033292].assignable

175324    1
Name: assignable, dtype: int32